In [1]:
import numpy as np
import matplotlib.pyplot as plt
from bokeh.io import output_notebook
from bokeh.layouts import row, column
from bokeh.plotting import show, figure
from bokeh.models import ColumnDataSource, Slider, CustomJS, LinearAxis, Range1d

output_notebook()

Loading BokehJS ...

In [2]:
def synapse_step(V_pre, V_post, params=dict(g_sat=0.15, k=0.05, n=1, t_delay=1e-4, V_th=-50.5, V_rev=-50)):
    g_sat = params['g_sat']
    k = params['k']
    n = params['n']
    t_delay = params['t_delay']
    V_th = params['V_th']
    V_rev = params['V_rev']
    g = np.minimum(g_sat, k*np.maximum((V_pre-V_th)**n, 0))
    I_syn = g*(V_post - V_rev)
    
    return I_syn

In [3]:
def ml_step(I_syn, V, N, params=dict(V_1=[30.0], V_2=[15.0], V_3=[0.0], V_4=[30.0], 
                                    phi=[0.025], C=[6.698], dt=[1e-4],
                                    E_L=[-50.0], E_Ca=[100.0], E_K=[-70.0],
                                    g_L=[0.5], g_Ca=[1.1], g_K=[2.0])):
    dt = params['dt'][0]*1e3
    V_1 = params['V_1'][0]
    V_2 = params['V_2'][0]
    V_3 = params['V_3'][0]
    V_4 = params['V_4'][0]
    phi = params['phi'][0]
    E_L = params['E_L'][0]
    E_Ca = params['E_Ca'][0]
    E_K = params['E_K'][0]

    g_L = params['g_L'][0]
    g_Ca = params['g_Ca'][0]
    g_K = params['g_K'][0]
    C = params['C'][0]
    
    dV = (I_syn - g_L*(V-E_L) - 0.5*g_Ca*(1+np.tanh((V-V_1)/V_2))*(V-E_Ca) - g_K*N*(V-E_K))/C
    dN = (0.5*(1+np.tanh((V-V_3)/V_4))-N)*(phi*np.cosh((V-V_3)/(2*V_4)))

    V1 = V + dV*dt
    N1 = N + dN*dt
    return [V1, N1]

In [18]:
def bkapp(doc):
    dt = 1e-4
    t = np.arange(0, 1, dt)
    
    V_init=-53.
    N_init=0.02
    
    V = dict(pre=np.zeros_like(t), post=np.zeros_like(t))
    for key, value in V.items():
        value[0] = V_init

    N = dict(pre=np.zeros_like(t), post=np.zeros_like(t))
    for key, value in N.items():
        value[0] = N_init
    
    I_ext = np.zeros_like(t)
    I_syn = np.zeros_like(t)
    I_ext[2000:3000] = 5
    
    
    for i in range(len(t)):
        I_syn[i] = synapse_step(V['pre'][i], V['post'][i])
        V_pre_tmp, N_pre_tmp = ml_step(I_ext[i], V['pre'][i], N['pre'][i])
        V_post_tmp, N_post_tmp = ml_step(I_syn[i], V['post'][i], N['post'][i])
        if i < len(t)-1:
            V['pre'][i+1] = V_pre_tmp
            N['pre'][i+1] = N_pre_tmp
            V['post'][i+1] = V_post_tmp
            N['post'][i+1] = N_post_tmp
    
    
    source = ColumnDataSource(data=dict(t=t, V_pre=V['pre'], V_post=V['post'], I_syn=I_syn, I_ext=I_ext))
    p1 = figure(width=600, height=300)
    p1.line('t', 'V_pre', source=source, line_color='skyblue', legend_label='V_pre')
    p1.line('t', 'V_post', source=source, line_color='orange', legend_label='V_post')
    p1.legend.click_policy='hide'
    p2 = figure(width=600, height=300)
    p2.line('t', 'I_ext', source=source, line_color='skyblue', legend_label='I_ext')
    p2.line('t', 'I_syn', source=source, line_color='orange', legend_label='I_syn')
    p2.legend.click_policy='hide'
    
    g_slider = Slider(value=0.15, start=0.01, end=1.0, step=0.01, title='g_sat', format='0.000')
    k_slider = Slider(value=0.05, start=0.01, end=1.0, step=0.01, title='k', format='0.000')
    n_slider = Slider(value=1, start=1, end=10, step=1, title='n')
    th_slider = Slider(value=-50.5, start=-100.0, end=0.0, step=0.5, title='V_th')
    rev_slider = Slider(value=-50.0, start=-100.0, end=0.0, step=0.5, title='V_rev')
    
    def update(attrname, old, new):
        g_sat = g_slider.value
        k = k_slider.value
        n = n_slider.value
        V_th = th_slider.value
        V_rev = rev_slider.value
        
        syn_params=dict(g_sat=g_sat, k=k, n=n, t_delay=1e-4, V_th=V_th, V_rev=V_rev)
        
        V = dict(pre=np.zeros_like(t), post=np.zeros_like(t))
        for key, value in V.items():
            value[0] = V_init

        N = dict(pre=np.zeros_like(t), post=np.zeros_like(t))
        for key, value in N.items():
            value[0] = N_init
            
        I_ext = np.zeros_like(t)
        I_syn = np.zeros_like(t)
        I_ext[2000:3000] = 5
        
        for i in range(len(t)):
            I_syn[i] = synapse_step(V['pre'][i], V['post'][i], syn_params)
            V_pre_tmp, N_pre_tmp = ml_step(I_ext[i], V['pre'][i], N['pre'][i])
            V_post_tmp, N_post_tmp = ml_step(I_syn[i], V['post'][i], N['post'][i])
            if i < len(t)-1:
                V['pre'][i+1] = V_pre_tmp
                N['pre'][i+1] = N_pre_tmp
                V['post'][i+1] = V_post_tmp
                N['post'][i+1] = N_post_tmp
                
        source.data = dict(t=t, V_pre=V['pre'], V_post=V['post'], I_syn=I_syn, I_ext=I_ext)
                              
    for w in [g_slider, k_slider, n_slider, th_slider, rev_slider]:
        w.on_change('value_throttled', update)
                              
    inputs = column(g_slider, k_slider, n_slider, th_slider, rev_slider)
    
    plots = column(p1, p2)
    
    doc.add_root(row(inputs, plots))

In [19]:
show(bkapp, notebook_url="http://localhost:8888")